# Setup

In [1]:
%reload_ext autoreload

In [2]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
np.set_printoptions(precision=3)
import scipy.stats
import einops


from functools import partial
from collections import namedtuple
import itertools
from itertools import repeat
from time import time

import chex
import jax
import jax.random as jr
import jax.numpy as jnp
from jax import vmap, grad, jit, lax
from jax import numpy as jnp
import jax.scipy as jsp

from flax.core import freeze, unfreeze
from flax import linen as nn
import flax

import jaxopt
import optax


from PIL import Image


#jax.config.update("jax_enable_x64", False)



In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds


import torch
from torch.utils.data import TensorDataset
import torchvision.transforms as T

In [2]:
import os 
cpu_count = os.cpu_count()
print(cpu_count)

# Run jax on multiple CPU cores
# https://github.com/google/jax/issues/5506
# https://stackoverflow.com/questions/72328521/jax-pmap-with-multi-core-cpu
import os 
#os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=90'

import jax
print(jax.devices())

96
[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(0,0,0), core_on_chip=1), TpuDevice(id=2, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,0,0), core_on_chip=1), TpuDevice(id=4, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=5, process_index=0, coords=(0,1,0), core_on_chip=1), TpuDevice(id=6, process_index=0, coords=(1,1,0), core_on_chip=0), TpuDevice(id=7, process_index=0, coords=(1,1,0), core_on_chip=1)]


# Load the labels

In [100]:
%pwd

'/home/kpmurphy/github/label-shift/tta'

In [9]:
from pathlib import Path
root = '/home/kpmurphy/data/CheXpert'
root = Path(root)
labels = pd.read_csv(root / "labels.csv", index_col="image_id")




In [12]:
print(len(labels.columns))
print(labels.columns)


21
Index(['Unnamed: 0', 'NO_FINDING', 'ENLARGED_CARDIOMEDIASTINUM',
       'CARDIOMEGALY', 'AIRSPACE_OPACITY', 'LUNG_LESION', 'PULMONARY_EDEMA',
       'CONSOLIDATION', 'PNEUMONIA', 'ATELECTASIS', 'PNEUMOTHORAX', 'EFFUSION',
       'PLEURAL_OTHER', 'FRACTURE', 'SUPPORT_DEVICES', 'patient_id', 'split',
       'GENDER', 'AGE_AT_CXR', 'PRIMARY_RACE', 'ETHNICITY'],
      dtype='object')


In [3]:

labels.head()

,Unnamed: 0,NO_FINDING,ENLARGED_CARDIOMEDIASTINUM,CARDIOMEGALY,AIRSPACE_OPACITY,LUNG_LESION,PULMONARY_EDEMA,CONSOLIDATION,PNEUMONIA,ATELECTASIS,...,EFFUSION,PLEURAL_OTHER,FRACTURE,SUPPORT_DEVICES,patient_id,split,GENDER,AGE_AT_CXR,PRIMARY_RACE,ETHNICITY
image_id,,,,,,,,,,,,,,,,,,,,,
CheXpert-v1.0/train/patient42720/study2/view1_frontal.jpg,0,3,1,3,3,3,1,3,3,3,...,1,3,3,1,patient42720,train,Male,58,White,Non-Hispanic/Non-Latino
CheXpert-v1.0/train/patient42720/study7/view1_frontal.jpg,1,3,3,0,1,3,3,3,3,3,...,1,3,3,1,patient42720,train,Male,58,White,Non-Hispanic/Non-Latino
CheXpert-v1.0/train/patient42720/study8/view1_frontal.jpg,2,3,3,0,1,3,3,3,3,3,...,1,3,3,1,patient42720,train,Male,58,White,Non-Hispanic/Non-Latino
CheXpert-v1.0/train/patient42720/study6/view1_frontal.jpg,3,3,3,3,3,3,1,3,3,3,...,1,3,3,1,patient42720,train,Male,58,White,Non-Hispanic/Non-Latino
CheXpert-v1.0/train/patient42720/study1/view1_frontal.jpg,4,3,3,3,3,3,1,3,3,3,...,1,3,3,1,patient42720,train,Male,58,White,Non-Hispanic/Non-Latino


In [27]:
len(labels)

190499

# Load joined data

(created by matching.py)

In [3]:
from pathlib import Path
root = '/home/kpmurphy/data/CheXpert'
root = Path(root)

data = np.load(root / 'data_matrix.npz', allow_pickle=True)


In [4]:
data.files

['features', 'attributes', 'columns', 'uniques']

In [8]:
print(data['features'].shape, data['attributes'].shape)

(190499, 1376) (190499, 19)


In [13]:
print(len(data['columns']))
print(data['columns'])


19
['NO_FINDING' 'ENLARGED_CARDIOMEDIASTINUM' 'CARDIOMEGALY'
 'AIRSPACE_OPACITY' 'LUNG_LESION' 'PULMONARY_EDEMA' 'CONSOLIDATION'
 'PNEUMONIA' 'ATELECTASIS' 'PNEUMOTHORAX' 'EFFUSION' 'PLEURAL_OTHER'
 'FRACTURE' 'SUPPORT_DEVICES' 'split' 'GENDER' 'AGE_AT_CXR' 'PRIMARY_RACE'
 'ETHNICITY']


In [7]:
print(data['uniques'])

{'split': Index(['train', 'valid'], dtype='object'), 'GENDER': Index(['Female', 'Male', 'Unknown'], dtype='object'), 'PRIMARY_RACE': Index(['American Indian or Alaska Native', 'Asian', 'Asian - Historical Conv',
       'Asian, Hispanic', 'Asian, non-Hispanic', 'Black or African American',
       'Black, Hispanic', 'Black, non-Hispanic', 'Native American, Hispanic',
       'Native American, non-Hispanic',
       'Native Hawaiian or Other Pacific Islander', 'Other', 'Other, Hispanic',
       'Other, non-Hispanic', 'Pacific Islander, Hispanic',
       'Pacific Islander, non-Hispanic', 'Patient Refused',
       'Race and Ethnicity Unknown', 'Unknown', 'White', 'White or Caucasian',
       'White, Hispanic', 'White, non-Hispanic'],
      dtype='object'), 'ETHNICITY': Index(['Hispanic', 'Hispanic/Latino', 'Non-Hispanic/Non-Latino',
       'Not Hispanic', 'Patient Refused', 'Unknown'],
      dtype='object')}


In [48]:

disease_ndx = np.where(data['columns'] == 'EFFUSION')[0][0]
Yall = data['attributes'][:, disease_ndx]

gender_ndx = np.where(data['columns'] == 'GENDER')[0][0]
Zall = data['attributes'][:, gender_ndx]

# diease labels
# 0 = no mention  
# 1 = positive   
# 2 = uncertain  
# 3 = negative  
# gender: 0=female, 1= male, 2=unknown
print('number of unknown gender = ', np.sum(Zall==2))
print('number of unknown disease = ', np.sum( (Yall==0) | (Yall==2) ) )
remove = (Yall==0) | (Yall==2) | (Zall==2)
ndx = np.where(remove==True)[0]
nremove = len(ndx)
print('removing ', nremove, ' rows')
keep = ~remove

X = data['features'][keep]

Y = Yall[keep]
pos = (Y==1); neg = (Y==3)
Y[pos]=1; Y[neg]=0

Z  = Zall[keep]

print(X.shape, Y.shape, Z.shape)




number of unknown gender =  1
number of unknown disease =  34898
removing  34898  rows
(155601, 1376) (155601,) (155601,)


In [49]:
print(np.unique(Y), np.unique(Z))

[0 1] [0 1]
